In [1]:
import pandas as pd
import numpy as np
from pandas_ml import ConfusionMatrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
df = pd.read_csv('Data/data_task3.csv', sep='\t')
pd.options.display.max_rows = 10

In [3]:
# количество асессоров
un = df.login.dropna().unique()
len(un)

600

In [4]:
tasks_d = []
def tasks_count():  
    for login in un:
        worker = df['login'] == login
        df_worker = df[worker]
        tasks = df_worker.docid.count()
        tasks_d.append({'login': login, 'tasks': tasks})

In [5]:
#количество заданий, выполненных каждым асессором
tasks_count()
tasks_done = pd.DataFrame(tasks_d)
tasks_done.sort_values(by=['tasks'])

,login,tasks
598,assessor234,99
561,assessor550,128
553,assessor255,364
359,assessor202,365
540,assessor420,365
...,...,...
550,assessor165,467
344,assessor368,467
524,assessor161,480
214,assessor140,481


In [6]:
#распределение правильных ответов по классам
df.cjud.value_counts()

0    220020
1     29980
Name: cjud, dtype: int64

Асессоры выполнили разное число заданий. Если один из них условно сделал два задания и ошибся в одном, его точность будет 0.5.
<br>
Если другой асессор сделал четыреста заданий и ошибся в двухстах, его точность тоже составит 0.5, но кажется, что второй асессор работает хуже.
<br>
Правильные ответы в наборе не сбалансированы по классам, поэтому если асессор будет бездумно проставлять 0 во всех заданиях, его точность составит 0.86.
<br>
С учетом обоих наблюдений показатель точности как количество правильных ответов к общему числу выполненных заданий не является адекватной метрикой.
<br>
В случае неизвестной природы оценок и несбалансированных классов эффективной метрикой может послужить **коэффициент Мэтьюса** (MCC), который, как и точность, выводится из матрицы ошибок.

Если TP - правильно установленная 1, FP - ошибочная 1, TN - правильно установленный 0, а FN - ошибочный 0, то матрица ошибок:

|-     | cjud | cjud |
|------|------|------|
| jud  |  TP  |  FP  |
| jud  |  FN  |  TN  |

Точность = (TP + TN) / (TP + FP + TN + FN)
<br>
**Коэффициент Мэтьюса** = (TP * TN - FP * FN) / √(TP + FP)(TP + FN)(TN + FP)(TN + FN)

Этот коэффициент будет высоким, только если асессор хорошо оценивает и 0, и 1 и исключает возможность бездумного проставления одинаковых оценок, поскольку тогда для оценки 0 TP и FN будут равны нулю и коэффициент Мэтьюса будет неопределенным.

In [7]:
matthews = []
def mathews_cc():
    for login in un:
        worker = df['login'] == login
        df_worker = df[worker]
        y_true = df_worker.cjud
        y_pred = df_worker.jud
        cm = ConfusionMatrix(y_true, y_pred)
        mcc = cm.MCC
        matthews.append({'login': login, 'MCC': mcc})

In [8]:
#Коэффициент Мэтьюса
mathews_cc()
workers_mcc = pd.DataFrame(matthews)
workers_mcc.sort_values(by=['MCC'])

,MCC,login
532,-0.053279,assessor56
299,-0.023192,assessor3
203,0.008214,assessor163
598,0.029355,assessor234
285,0.039235,assessor390
...,...,...
253,0.716138,assessor151
77,0.716357,assessor596
125,0.724209,assessor450
491,0.727975,assessor82


Также можно сравнить **сбалансированную точность** (TP / (TP + TN) + TN / (TN + FN)) / 2 

In [9]:
balanced = []
def bal_acc():
    for login in un:
        worker = df['login'] == login
        df_worker = df[worker]
        y_true = df_worker.cjud
        y_pred = df_worker.jud
        ba = balanced_accuracy_score(y_true, y_pred)
        balanced.append({'login': login, 'b. acc': ba})

In [10]:
#Сбалансированная точность
bal_acc()
workers_bac = pd.DataFrame(balanced)
workers_bac.sort_values(by=['b. acc'])

,b. acc,login
532,0.458219,assessor56
299,0.481892,assessor3
203,0.504155,assessor163
598,0.521008,assessor234
285,0.530749,assessor390
...,...,...
491,0.934404,assessor82
280,0.934625,assessor38
541,0.935958,assessor458
370,0.940381,assessor259


Аналогичный результат в данном случае показывает метрика **ROC AUC**, которая считает площадь под кривой качества классификации. 
<br>
Чем больше площадь между кривой ROC и осью, которая тождественна гаданию с вероятностью 0.5, тем лучше классификация. 


In [11]:
rocauc = []
def roc_auc():
    for login in un:
        worker = df['login'] == login
        df_worker = df[worker]
        y_true = df_worker.cjud
        y_scores = df_worker.jud
        ra = roc_auc_score(y_true, y_scores)
        rocauc.append({'login': login, 'roc_auc': ra})

In [12]:
#ROC AUC
roc_auc()
workers_rocauc = pd.DataFrame(rocauc)
workers_rocauc = workers_rocauc[['roc_auc','login']]
workers_rocauc.sort_values(by=['roc_auc'])

,roc_auc,login
532,0.458219,assessor56
299,0.481892,assessor3
203,0.504155,assessor163
598,0.521008,assessor234
285,0.530749,assessor390
...,...,...
491,0.934404,assessor82
280,0.934625,assessor38
541,0.935958,assessor458
370,0.940381,assessor259


Если природа оценок известна, то на первое место могут выйти другие показатели, например, **точность (precision)** - сколько элементов из распознанных являются релевантными, и **полнота (recall, specificity)** - сколько релевантных элементов из их общего числа удалось распознать.
<br>
**precision** = TP / (TP + FP)
<br>
**recall** = TP / (TP + FN)
<br>
В зависимости от природы оценок может понадобиться и **специфичность (specificity)** - доля отрицательных элементов, которые удалось распознать.
<br>
**specificity** = TN (TN + FP)

Например, если службе безопасности важно не пропустить ни одного террориста и 1 - человек является террористом, то важнее полнота.
<br>
Если надо найти релевантные запросу страницы, то если для него нет страниц, которые обязательно должны попасть в выдачу, тогда более важной будет точность.

In [13]:
#Точность и полнота
precrec = []
def prec_rec():
    for login in un:
        worker = df['login'] == login
        df_worker = df[worker]
        y_true = df_worker.cjud
        y_pred = df_worker.jud
        prec = precision_score(y_true, y_pred)
        rec = recall_score(y_true, y_pred)
        precrec.append({'login': login, 'precision': prec, 'recall': rec})

In [14]:
prec_rec()
workers_precrec = pd.DataFrame(precrec)
workers_precrec.sort_values(by=['precision'])

,login,precision,recall
532,assessor56,0.097458,0.500000
203,assessor163,0.102564,0.105263
299,assessor3,0.108225,0.510204
501,assessor118,0.112150,0.615385
285,assessor390,0.125561,0.595745
...,...,...,...
199,assessor487,0.631579,0.923077
226,assessor565,0.643678,0.811594
552,assessor577,0.646341,0.868852
348,assessor462,0.655172,0.826087
